## Deliverable 2. Create a Customer Travel Destinations Map.

In [5]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [8]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Nizhnyaya Tavda,RU,2021-09-17 12:12:29,57.6731,66.1725,39.42,97,100,14.45,overcast clouds
1,1,Shizunai,JP,2021-09-17 12:09:52,42.3339,142.3669,62.56,88,100,15.50,light rain
2,2,Gat,IL,2021-09-17 12:12:30,31.6100,34.7642,89.40,41,0,14.99,clear sky
3,3,Boyolangu,ID,2021-09-17 12:12:30,-8.1181,111.8935,75.81,90,97,5.99,overcast clouds
4,4,Tevaitoa,PF,2021-09-17 12:12:30,-16.7833,-151.5000,78.35,74,9,25.28,clear sky
5,5,Narsaq,GL,2021-09-17 12:12:31,60.9167,-46.0500,39.42,72,89,2.68,overcast clouds
6,6,Port Elizabeth,ZA,2021-09-17 12:12:31,-33.9180,25.5701,65.10,56,76,10.87,broken clouds
7,7,Barrow,US,2021-09-17 12:12:32,71.2906,-156.7887,33.82,93,90,3.44,mist
8,8,Dikson,RU,2021-09-17 12:12:32,73.5069,80.5464,33.89,71,100,17.60,overcast clouds
9,9,Novorzhev,RU,2021-09-17 12:12:32,57.0294,29.3338,54.55,47,53,10.31,broken clouds


In [9]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))


What is the minimum temperature you would like for your trip? 71
What is the maximum temperature you would like for your trip? 89


In [10]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
cities_preferred_df = city_data_df.loc[(city_data_df["Max Temp"] >= min_temp) & (city_data_df["Max Temp"] <= max_temp)]

cities_preferred_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
3,3,Boyolangu,ID,2021-09-17 12:12:30,-8.1181,111.8935,75.81,90,97,5.99,overcast clouds
4,4,Tevaitoa,PF,2021-09-17 12:12:30,-16.7833,-151.5000,78.35,74,9,25.28,clear sky
10,10,Faanui,PF,2021-09-17 12:12:33,-16.4833,-151.7500,78.55,73,7,24.56,light rain
13,13,Galle,LK,2021-09-17 12:12:34,6.0367,80.2170,81.55,82,72,10.94,broken clouds
20,20,Pontianak,ID,2021-09-17 12:08:39,-0.0333,109.3333,82.38,83,75,0.00,thunderstorm


In [11]:
# 4a. Determine if there are any empty rows.
cities_preferred_df.isnull().count()

City_ID                303
City                   303
Country                303
Date                   303
Lat                    303
Lng                    303
Max Temp               303
Humidity               303
Cloudiness             303
Wind Speed             303
Current Description    303
dtype: int64

In [12]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = cities_preferred_df.dropna()
clean_df.isnull().sum()

City_ID                0
City                   0
Country                0
Date                   0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [13]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Boyolangu,ID,75.81,overcast clouds,-8.1181,111.8935,
4,Tevaitoa,PF,78.35,clear sky,-16.7833,-151.5000,
10,Faanui,PF,78.55,light rain,-16.4833,-151.7500,
13,Galle,LK,81.55,broken clouds,6.0367,80.2170,
20,Pontianak,ID,82.38,thunderstorm,-0.0333,109.3333,
27,Davila,PH,83.91,light rain,18.4726,120.5763,
28,Bengkulu,ID,80.26,broken clouds,-3.8004,102.2655,
29,Hilo,US,73.80,light rain,19.7297,-155.0900,
30,Itoman,JP,82.42,few clouds,26.1247,127.6694,
31,Belmonte,BR,79.30,clear sky,-15.8631,-38.8828,


In [14]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found...skipping the city.")
        

Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.


In [15]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()

In [16]:
# 8a. Create the output File (CSV)
output_data_file = "Weather_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [17]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [18]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.75)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))